In [2]:
import pandas as pd
import json
import websocket

In [18]:
endpoint = "wss://stream.binance.com:9443/ws"

request = json.dumps({
    'method': 'SUBSCRIBE',
    'params': ['ethusdt@miniTicker'],
    'id':1
})

In [21]:
df = pd.DataFrame()
position = False
buy_orders, sell_orders = [], []

def on_open(ws):
    ws.send(request)

def on_message(ws, message):
    global df, position, buy_orders, sell_orders

    out = json.loads(message)
    out = pd.DataFrame({'price': float(out['c'])},
                       index=[pd.to_datetime(out['E'], 
                                             unit='ms')])
    
    df = pd.concat([df,out], axis=0)
    # print(df)
    
    df = df.tail(7) # so that memory doesn't crash
    last_price = df.tail(1).price.values[0]
    sma5 = df.price.rolling(5).mean().tail(1).values[0]
    # print(f"last price is {last_price}, sma5 is {sma5}")
    
    # trading logic:
    if not position and last_price > sma5:
        buy_price = last_price
        print(f"bought for {last_price}")
        buy_orders.append(last_price)
        position = True
    
    if position and last_price < sma5:
        print(f"sold for {last_price}")
        sell_orders.append(last_price)
        position = False

In [22]:
ws = websocket.WebSocketApp(endpoint,
                            on_message=on_message,
                            on_open=on_open)

ws.run_forever()

bought for 2148.5
sold for 2148.49
bought for 2148.5
sold for 2148.3
bought for 2148.45
sold for 2149.29


True